In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#PCA, Resampling, and Random Forest

In [ ]:
import pandas as pd
csvfile = '/content/drive/My Drive/Colab Notebooks/Seagate-4.24.2.csv'
df = pd.read_csv(csvfile, encoding='latin1')

In [ ]:
df2 = df[['mtrprm3_parameter4','mtrprm9_parameter2','mtrprm11_parameter1','mtrprm19_parameter1','mtrprm20_parameter2','mtrprm21_parameter2','mtrprm24_parameter4',
   'hkpinp2_parameter1','hkpinp4_parameter18','Target','drive_interface','drive_product_group','motor_vend_id','run_type']]

In [ ]:
df2 = pd.get_dummies(df2, columns=['drive_product_group'], prefix = ['drive_product_group'])

df2 = pd.get_dummies(df2, columns=['drive_interface'], prefix = ['drive_interface'])

df2 = pd.get_dummies(df2, columns=['motor_vend_id'], prefix = ['motor_vend_id'])

df2 = pd.get_dummies(df2, columns=['run_type'], prefix = ['run_type'])

hkpinp4_parameter18': 0.13724996480483434, 'hkpinp2_parameter1': 0.13001423837115028, 'mtrprm19_parameter1': 0.12530402567772822, 'mtrprm9_parameter2': 0.12457584504842446, 'mtrprm11_parameter1': 0.12412212664851024,

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import metrics
from IPython.display import Image  
from pydotplus import graph_from_dot_data

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df2,test_size=0.3)

In [ ]:
# Separate target and predictors
y_train = train['Target']
x_train = train.drop(['Target'], axis=1)
y_test = test['Target']
x_test = test.drop(['Target'], axis=1)
#y_validate = validate['Target']
#x_validate = validate.drop(['Target'], axis=1)

# Check the balance of the splits on y_

In [ ]:
# Applying PCA function on training 
# and testing set of X component 
from sklearn.decomposition import PCA 
  
pca = PCA(n_components = 5) 
  
x_train = pca.fit_transform(x_train) 
x_test = pca.transform(x_test) 
  
explained_variance = pca.explained_variance_ratio_

In [ ]:
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import EditedNearestNeighbours

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from imblearn.under_sampling import NearMiss 
nr = NearMiss() 

X_train_miss, y_train_miss = nr.fit_sample(x_train,y_train.ravel())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from collections import Counter
ada = ADASYN(random_state=42)
X_resampled, y_resampled = ada.fit_resample(x_train, y_train)
print('Resampled dataset after ADASYN shape %s' % Counter(y_resampled))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Resampled dataset after ADASYN shape Counter({1: 196426, 0: 196173})


In [ ]:
y_test.sum()

716

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Instantiate
svm = SVC(probability = True)
# Fit
svm_model = svm.fit(X_train_miss, y_train_miss)

svm_model.score(X_train_miss, y_train_miss)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(svm_model.predict(x_test))
probs = pd.DataFrame(svm_model.predict_proba(x_test))

# Store metrics
svm_accuracy = metrics.accuracy_score(y_test, predicted)     
svm_roc_auc = metrics.roc_auc_score(y_test, probs[1])       
svm_confus_matrix = metrics.confusion_matrix(y_test, predicted) 
svm_classification_report = metrics.classification_report(y_test, predicted)
svm_precision = metrics.precision_score(y_test, predicted, pos_label=1)
svm_recall = metrics.recall_score(y_test, predicted, pos_label=1)
svm_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation 
svm_cv_scores = cross_val_score(SVC(), x_test, y_test, scoring='precision', cv=10)
svm_cv_mean = np.mean(svm_cv_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined a

rf_confus_matrix

In [ ]:
svm_confus_matrix

array([[ 3781, 80331],
       [   19,   697]])

In [ ]:
svm_accuracy

0.05278917338614608

In [ ]:
svm_recall

0.973463687150838

In [ ]:
svm_precision

0.008601964752924915

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.svm import SVC as svc 
from sklearn.metrics import make_scorer, roc_auc_score
from scipy import stats

In [ ]:
# RANDOM SEARCH FOR 20 COMBINATIONS OF PARAMETERS
rand_list = {"C": stats.uniform(2, 10),
             "gamma": stats.uniform(0.1, 1)}
              
rand_search = RandomizedSearchCV(svm_model, param_distributions = rand_list, n_iter = 20, n_jobs = 4, cv = 3) 
rand_search.fit(x_train, y_train) 
rand_search.cv_results_

In [ ]:
svm_confus_matrix

array([[ 3366, 80742],
       [   15,   705]])

In [ ]:
svm_precision

0.008655935761906515

In [ ]:
svm_recall

0.9791666666666666

In [ ]:
svm_roc_auc

0.534663905996523

In [ ]:
from collections import Counter
Counter(y_train)
Counter(y_test)

Counter({0: 84108, 1: 720})

In [ ]:
# Instantiate
svm = SVC(probability = True)
# Fit
svm_model = svm.fit(X_resampled, y_resampled)

svm_model.score(X_resampled, y_resampled)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(svm_model.predict(x_test))
probs = pd.DataFrame(svm_model.predict_proba(x_test))

# Store metrics
svm_accuracy = metrics.accuracy_score(y_test, predicted)     
svm_roc_auc = metrics.roc_auc_score(y_test, probs[1])       
svm_confus_matrix = metrics.confusion_matrix(y_test, predicted) 
svm_classification_report = metrics.classification_report(y_test, predicted)
svm_precision = metrics.precision_score(y_test, predicted, pos_label=1)
svm_recall = metrics.recall_score(y_test, predicted, pos_label=1)
svm_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation 
svm_cv_scores = cross_val_score(SVC(probability=True), x_test, y_test, scoring='precision', cv=10)
svm_cv_mean = np.mean(svm_cv_scores)

In [ ]:
svm_confus_matrix

In [ ]:
svm_accuracy

In [ ]:
svm_recall

In [ ]:
svm_roc_auc

In [ ]:
# Instantiate
rf = RandomForestClassifier(random_state = 1, max_depth = 15, n_estimators = 700, min_samples_split = 0.5, min_samples_leaf = 0.3,max_features=3)
# Fit
rf_model = rf.fit(X_train_miss, y_train_miss)

rf_model.score(X_train_miss, y_train_miss)

# Predictions/probs on the test dataset
predicted = pd.DataFrame(rf_model.predict(x_test))
probs = pd.DataFrame(rf_model.predict_proba(x_test))

# Store metrics
rf_accuracy = metrics.accuracy_score(y_test, predicted)     
rf_roc_auc = metrics.roc_auc_score(y_test, probs[1])       
rf_confus_matrix = metrics.confusion_matrix(y_test, predicted) 
rf_classification_report = metrics.classification_report(y_test, predicted)
rf_precision = metrics.precision_score(y_test, predicted, pos_label=1)
rf_recall = metrics.recall_score(y_test, predicted, pos_label=1)
rf_f1 = metrics.f1_score(y_test, predicted, pos_label=1)

# Evaluate the model using 10-fold cross-validation 
rf_cv_scores = cross_val_score(RandomForestClassifier(), x_test, y_test, scoring='precision', cv=10)
rf_cv_mean = np.mean(rf_cv_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined a

In [ ]:
rf_confus_matrix

array([[45116, 39010],
       [  267,   435]])

In [ ]:
rf_recall

0.6196581196581197